### Setup Environment

In [4]:
import os
import re
import math
import json
from dotenv import load_dotenv

from groq import Groq

from agentic_patterns.tool_pattern.tool import tool
from agentic_patterns.utils.extraction import extract_tag_content


# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

MODEL = "llama-3.3-70b-versatile"
GROQ_CLIENT = Groq()

In [5]:
@tool
def sum_two_elements(a: int, b: int) -> int:
    """
    Computes the sum of two integers.

    Args:
        a (int): The first integer to be summed.
        b (int): The second integer to be summed.

    Returns:
        int: The sum of `a` and `b`.
    """
    return a + b


@tool
def multiply_two_elements(a: int, b: int) -> int:
    """
    Multiplies two integers.

    Args:
        a (int): The first integer to multiply.
        b (int): The second integer to multiply.

    Returns:
        int: The product of `a` and `b`.
    """
    return a * b

@tool
def compute_log(x: int) -> float | str:
    """
    Computes the logarithm of an integer `x` with an optional base.

    Args:
        x (int): The integer value for which the logarithm is computed. Must be greater than 0.

    Returns:
        float: The logarithm of `x` to the specified `base`.
    """
    if x <= 0:
        return "Logarithm is undefined for values less than or equal to 0."
    
    return math.log(x)


available_tools = {
    "sum_two_elements": sum_two_elements,
    "multiply_two_elements": multiply_two_elements,
    "compute_log": compute_log
}

In [6]:
# Define the System Prompt as a constant
REACT_SYSTEM_PROMPT = """
You are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.
You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don' make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.

For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools / actions:

<tools> 
%s
</tools>

Example session:

<question>What's the current temperature in Madrid?</question>
<thought>I need to get the current weather in Madrid</thought>
<tool_call>{"name": "get_current_weather","arguments": {"location": "Madrid", "unit": "celsius"}, "id": 0}</tool_call>

You will be called again with this:

<observation>{0: {"temperature": 25, "unit": "celsius"}}</observation>

You then output:

<response>The current temperature in Madrid is 25 degrees Celsius</response>

Additional constraints:

- If the user asks you something unrelated to any of the tools above, answer freely enclosing your answer with <response></response> tags.
"""

In [7]:
tools_signature = sum_two_elements.fn_signature + ",\n" + multiply_two_elements.fn_signature + ",\n" + compute_log.fn_signature
REACT_SYSTEM_PROMPT = REACT_SYSTEM_PROMPT % tools_signature

In [8]:
USER_QUESTION = "I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result"
chat_history = [
    {
        "role": "system",
        "content": REACT_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": f"<question>{USER_QUESTION}</question>"
    }
]

In [9]:
def ReAct(chat_history):
    for _ in range(10):

        output = GROQ_CLIENT.chat.completions.create(
        messages=chat_history,
        model=MODEL
    ).choices[0].message.content
        
        if "<response>" in output:
            print(output)
            break
            

        chat_history.append(
        {
            "role": "assistant",
            "content": output
        }
    )
        tool_call = extract_tag_content(output, tag="tool_call")
        tool_call = json.loads(tool_call.content[0])
        tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

        chat_history.append(
        {
            "role": "user",
            "content": f"<observation>{tool_result}</observation>"
        }
    )
    
    return chat_history
    
    
ReAct(chat_history)  
    

['{"name": "sum_two_elements", "arguments": {"a": 1234, "b": 5678}, "id": 0}']
['{"name": "multiply_two_elements", "arguments": {"a": 6912, "b": 5}, "id": 1}']
['{"name": "compute_log", "arguments": {"x": 34560}, "id": 2}']
<thought>I have the results of all the operations: the sum of 1234 and 5678 is 6912, multiplying this result by 5 gives 34560, and the logarithm of 34560 is approximately 10.45.</thought>
<response>The sum of 1234 and 5678 is 6912. Multiplying this result by 5 gives 34560. The logarithm of 34560 is approximately 10.45.</response>


[{'role': 'system',
  'content': '\nYou are a function calling AI model. You operate by running a loop with the following steps: Thought, Action, Observation.\nYou are provided with function signatures within <tools></tools> XML tags.\nYou may call one or more functions to assist with the user query. Don\' make assumptions about what values to plug\ninto functions. Pay special attention to the properties \'types\'. You should use those types as in a Python dict.\n\nFor each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:\n\n<tool_call>\n{"name": <function-name>,"arguments": <args-dict>, "id": <monotonically-increasing-id>}\n</tool_call>\n\nHere are the available tools / actions:\n\n<tools> \n{"name": "sum_two_elements", "description": "\\n    Computes the sum of two integers.\\n\\n    Args:\\n        a (int): The first integer to be summed.\\n        b (int): The second integer to be summed.\\n\\n    Returns:\\n   

### manually running each loop:

In [10]:
USER_QUESTION = "I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result"
chat_history = [
    {
        "role": "system",
        "content": REACT_SYSTEM_PROMPT
    },
    {
        "role": "user",
        "content": f"<question>{USER_QUESTION}</question>"
    }
]

In [11]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I need to calculate the sum of 1234 and 5678, then multiply the result by 5, and finally compute the logarithm of this result.</thought>
<tool_call>{"name": "sum_two_elements", "arguments": {"a": 1234, "b": 5678}, "id": 0}</tool_call>


In [12]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

In [13]:
tool_call = extract_tag_content(output, tag="tool_call")

['{"name": "sum_two_elements", "arguments": {"a": 1234, "b": 5678}, "id": 0}']


In [14]:
tool_call = json.loads(tool_call.content[0])

In [15]:
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])


In [16]:
assert tool_result == 1234 + 5678

In [17]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

In [18]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I have the sum of 1234 and 5678, which is 6912. Now, I need to multiply this result by 5.</thought>
<tool_call>{"name": "multiply_two_elements", "arguments": {"a": 6912, "b": 5}, "id": 1}</tool_call>


In [19]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

In [20]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

['{"name": "multiply_two_elements", "arguments": {"a": 6912, "b": 5}, "id": 1}']


In [21]:
assert tool_result == (1234 + 5678) * 5

In [22]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

In [23]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I have the result of multiplying 6912 by 5, which is 34560. Now, I need to compute the logarithm of this result.</thought>
<tool_call>{"name": "compute_log", "arguments": {"x": 34560}, "id": 2}</tool_call>


In [24]:
chat_history.append(
    {
        "role": "assistant",
        "content": output
    }
)

In [25]:
tool_call = extract_tag_content(output, tag="tool_call")
tool_call = json.loads(tool_call.content[0])
tool_result = available_tools[tool_call["name"]].run(**tool_call["arguments"])

['{"name": "compute_log", "arguments": {"x": 34560}, "id": 2}']


In [26]:
assert tool_result == math.log((1234 + 5678) * 5)

In [27]:
chat_history.append(
    {
        "role": "user",
        "content": f"<observation>{tool_result}</observation>"
    }
)

In [28]:
output = GROQ_CLIENT.chat.completions.create(
    messages=chat_history,
    model=MODEL
).choices[0].message.content

print(output)

<thought>I have the logarithm of 34560, which is approximately 10.45. Now, I can provide the final result.</thought>
<response>The sum of 1234 and 5678 is 6912. Multiplying this result by 5 gives 34560. The logarithm of 34560 is approximately 10.45.</response>


### Scaling up: using ReAct arch

In [29]:
from agentic_patterns.planning_pattern.react_agent import ReactAgent

In [30]:
agent = ReactAgent(tools=[sum_two_elements, multiply_two_elements, compute_log])


In [31]:
agent.run(user_msg="I want to calculate the sum of 1234 and 5678 and multiply the result by 5. Then, I want to take the logarithm of this result")


[]
['I need to calculate the sum of 1234 and 5678, then multiply the result by 5, and finally compute the logarithm of this result.']
['{"name": "sum_two_elements","arguments": {"a": 1234, "b": 5678}, "id": 0}']

Thought: I need to calculate the sum of 1234 and 5678, then multiply the result by 5, and finally compute the logarithm of this result.

Using Tool: sum_two_elements

Tool call dict: 
{'name': 'sum_two_elements', 'arguments': {'a': 1234, 'b': 5678}, 'id': 0}

Tool result: 
6912

Observations: {0: 6912}
[]
['I have the sum of 1234 and 5678, which is 6912. Now, I need to multiply this result by 5.']
['{"name": "multiply_two_elements","arguments": {"a": 6912, "b": 5}, "id": 1}']

Thought: I have the sum of 1234 and 5678, which is 6912. Now, I need to multiply this result by 5.

Using Tool: multiply_two_elements

Tool call dict: 
{'name': 'multiply_two_elements', 'arguments': {'a': 6912, 'b': 5}, 'id': 1}

Tool result: 
34560

Observations: {1: 34560}
[]
['I have the result of mul

'The sum of 1234 and 5678 is 6912. Multiplying 6912 by 5 results in 34560. The logarithm of 34560 is approximately 10.45.'